In [1]:
import pandas as pd
import numpy as np
import os

path = os.getcwd()
df = pd.read_csv('dataset-final.csv')
print(path)

/home/baharali/Medical_Image/Implementations/2.BS3WC


In [2]:
df_ctg_05 = df[(df['File Name'] == 'Ctg-05') | (df['File Name'] == 'ctg-05')]
df_ctg_10 = df[(df['File Name'] == 'Ctg-10') | (df['File Name'] == 'ctg-10')]
df_ctg_15 = df[(df['File Name'] == 'Ctg-15') | (df['File Name'] == 'ctg-15')]
df_ctg_20 = df[(df['File Name'] == 'Ctg-20') | (df['File Name'] == 'ctg-20')]

df_pima_05 = df[df['File Name'] == 'pima-05']
df_pima_10 = df[df['File Name'] == 'pima-10']
df_pima_15 = df[df['File Name'] == 'pima-15']
df_pima_20 = df[df['File Name'] == 'pima-20']

In [3]:
df_ctg_05.shape
#df_pima_05[df_pima_05['Algo'] == '1-RE3WC']['Accuracy'].mean()
#df_pima_05[df_pima_05['Algo'] == '2-LOF']['Accuracy'].mean()
#df_pima_05[df_pima_05['Algo'] == '3-LoOP']['Accuracy'].std()

(100, 12)

In [4]:
df_ctg_20.shape
df_pima_20.shape
df_pima_20[df_pima_20['Algo'] == '2-LOF']['Accuracy'].std()

0.0030810752992878037

In [5]:
import csv

def write_to_file(file_name, values, header_names=False, mode='w'):
    with open(file_name, mode) as writeFile:
        writer = csv.writer(writeFile)
        if header_names and mode == 'w':
            writer.writerow(header_names)
        
        writer.writerows(values)
        

In [6]:
file_path = os.path.join(path, 'Tvalue_Pvalue.csv')
values = [['\\makecell{Comparing \\\\ Approach}', 'Scale', 't value', 'p value', 'Hypo', 'Mean1', 'Mean2', 'Mean Diff', 'Std1', 'Std2', 'Std Diff']]
write_to_file(file_path, np.array(values), mode='w')

In [7]:
def format_number(number, r=2):
    negative_flag = ''
    number_formated = ''
    if number < 0:
        negative_flag = '-'
        number = number*-1
        
    if int(number) >= 10000:
        number_formated = str(format(number,"10." + str(r) + "E"))
        number_formated = number_formated.replace('E+', ' \\times 10 ^ {') + '}'
    elif int(number * 10000) <= 0:
        number_formated = str(format(number,"10." + str(r) + "E"))
        number_formated = number_formated.replace('E-', ' \\times 10 ^ {-') + '}'
    elif int(number * 10) > 0:
        number_formated = str(round(number,r))
    elif int(number * 100) > 0:
        number_formated = str(round(number,r+1))
    elif int(number * 1000) > 0:
        number_formated = str(round(number,r+2))
    elif int(number * 10000) > 0:
        number_formated = str(round(number,r+3))
            
    number_formated = negative_flag + number_formated.strip()
    
    return number_formated

In [8]:
format_number(.19)

'0.19'

In [9]:
def check_statistical_significance(df, key1, key2, measurement_scale, flag=0):
    # Student's t-test
    from numpy.random import seed
    from numpy.random import randn
    from scipy.stats import ttest_ind
    
    values = []
    
    #Seed the random number generator
    seed(1)

    #Generate two independent samples
    data1 = np.array(df[df['Algo'] == key1][measurement_scale])
    data2 = np.array(df[df['Algo'] == key2][measurement_scale])
    
    data3 = []
    for i in range(len(data1)):
        data3.append(data1[i] - data2[i])
    
    
    
    mean1 = np.mean(data1)
    mean2 = np.mean(data2)
    
    std1 = np.std(data1)
    std2 = np.std(data2)
    
    std3 = np.std(data3)
    
    #Compare Samples
    stat, p = ttest_ind(data1, data2)
    
    algo_1 = key1.split('-')[1]
    algo_2 = key2.split('-')[1]
    
    #print('Statistics=%.3f, p=%.3f' % (stat, p))
    values.append([key1, key2, measurement_scale, format(stat,"10.2E"), format(p,"10.2E"), 'H1', format(mean1,"10.2E"), 
        format(mean2,"10.2E"), format((mean1-mean2),"10.2E"), format(std1,"10.2E"), format(std2,"10.2E"), format((std1-std2),"10.2E")])

    write_to_file(file_path, np.array(values), mode='a')
    
    
    #values2 = [[' & ' + key1 + ' & ' + key2 + ' & ' + measurement_scale + ' & ' + str(format(stat,"10.2E")) + ' & ' + 
    #    str(format(p,"10.2E")) + ' & ' + 'H1' + ' & ' + str(format(mean1,"10.2E")) + ' & ' + str(format(mean2,"10.2E")) + ' & ' + 
    #    str(format((mean1-mean2),"10.2E")) + ' & ' + str(format(std1,"10.2E")) + ' & ' + str(format(std2,"10.2E")) + r' \\']]
    
    if p < 0.05:
        if stat >= 0:
            hypo = '\\boldmath{$\mathrm{H_1}$}'
        else:
            hypo = '$\mathrm{H_1}$'
    else:
        hypo = '$\mathrm{H_0}$'
    
    p_formated = '$' + format_number(p) + '$'
    std3_formated = '$' + format_number(std3) + '$'
    stat_formated = '$' + format_number(stat) + '$'
    mean_formated = '$' + format_number(mean1-mean2) + '$'

    if flag:
        str_val = [[' & ' + algo_2 + ' & ' + stat_formated + ' & ' + 
                p_formated + ' & ' + hypo + ' & ' + mean_formated + ' & ' + 
                std3_formated + r' \\' + ' \\cline{3-8}']]
    else:
        str_val = [[' & ' + ' & ' + algo_2 + ' & ' + stat_formated + ' & ' + 
                p_formated + ' & ' + hypo + ' & ' + mean_formated + ' & ' + 
                std3_formated + r' \\' + ' \\cline{3-8}']]
    
    write_to_file(file_path2, np.array(str_val), mode='a')

    #Interpret
    alpha = 0.05
    if p > alpha:
        pass
        #print('Same distributions (fail to reject H0)')
    else:
        pass
        #print('Different distributions (reject H0)')

In [10]:
file_path2 = os.path.join(path, 'Tvalue_Pvalue_latex.txt')

def start_method(mode='w'):
    values = [[],[],
        ['\\begin{table}[ht!]'],
        ['\\centering'],
        ['\\resizebox{\linewidth}{!}{%'],
        ['\\begin{tabular}{| l | l | l | c | c | c | c | c |}'],
        ['\\hline &&&&&&& \\\\'],
        ['Dataset' + ' & ' + 'Measure' + ' & ' + '\\makecell{Comparing \\\\ Approach}' + ' & ' + '\\textit{t} value' + ' & ' + '\\textit{p} value' + ' & ' 
         + 'Hypo.' + ' & ' + 'Mean diff.' + ' & ' + 'Std. dev.' + r' \\' + ' \\hline'], 
        ['\\multirow{18}{*}{Cardiotocography}'],
        [' & ' + '\\multirow{9}{*}{Accuracy}'] ]
    
    
    write_to_file(file_path2, np.array(values), mode=mode)

def middle_method(mode='w'):
    values = [['\\hline'],
        ['\\multicolumn{8}{|c|}{} \\\\ [-2.5ex]  \\cline{1-8}'],
        ['\\multirow{18}{*}{Pima Diabetes}'],
        [' & ' + '\\multirow{9}{*}{Accuracy}'] ]
    
    write_to_file(file_path2, np.array(values), mode=mode)

def end_method(mode='w', tab_name='table', percentage='5'):
    values = [ ['\\hline'],
        #['\\multicolumn{7}{l}{H1: The mean difference is greater than zero.} \\\\'],
        ['\\hline'],
        ['\\end{tabular} }'],
        ['\\caption{Results of t-tests on datasets with ' + percentage + '\% downsampling}'],
        ['\\label{tab:' + tab_name + '}'],
        ['\\end{table}']]
    
    write_to_file(file_path2, np.array(values), mode=mode)



In [11]:
start_method(mode='w')

check_statistical_significance(df_ctg_05, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_ctg_05, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_ctg_05, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_ctg_05, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_ctg_05, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_ctg_05, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_ctg_05, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_ctg_05, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_ctg_05, '1-RE3WC', '10-AvgKNN', 'Accuracy')


values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]

write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_ctg_05, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_ctg_05, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_ctg_05, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '10-AvgKNN', 'F1-Score')


/home/baharali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [12]:
middle_method(mode='a')

In [13]:
check_statistical_significance(df_pima_05, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_pima_05, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_pima_05, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_pima_05, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_pima_05, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_pima_05, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_pima_05, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_pima_05, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_pima_05, '1-RE3WC', '10-AvgKNN', 'Accuracy')


values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')

check_statistical_significance(df_pima_05, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_pima_05, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_pima_05, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_pima_05, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_pima_05, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_pima_05, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_pima_05, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_pima_05, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_pima_05, '1-RE3WC', '10-AvgKNN', 'F1-Score')


end_method(mode='a', tab_name='table_05', percentage='5')

In [14]:
start_method(mode='a')

check_statistical_significance(df_ctg_10, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_ctg_10, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_ctg_10, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_ctg_10, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_ctg_10, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_ctg_10, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_ctg_10, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_ctg_10, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_ctg_10, '1-RE3WC', '10-AvgKNN', 'Accuracy')



values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_ctg_10, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_ctg_10, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_ctg_10, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_ctg_10, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_ctg_10, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_ctg_05, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_ctg_05, '1-RE3WC', '10-AvgKNN', 'F1-Score')



/home/baharali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [15]:
middle_method(mode='a')

In [16]:

check_statistical_significance(df_pima_10, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_pima_10, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_pima_10, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_pima_10, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_pima_10, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_pima_10, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_pima_10, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_pima_10, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_pima_10, '1-RE3WC', '10-AvgKNN', 'Accuracy')


values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_pima_10, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_pima_10, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_pima_10, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_pima_10, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_pima_10, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_pima_10, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_pima_10, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_pima_10, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_pima_10, '1-RE3WC', '10-AvgKNN', 'F1-Score')


end_method(mode='a', tab_name='table_10', percentage='10')


In [17]:
start_method(mode='a')

check_statistical_significance(df_ctg_15, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_ctg_15, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_ctg_15, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_ctg_15, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_ctg_15, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_ctg_15, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_ctg_15, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_ctg_15, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_ctg_15, '1-RE3WC', '10-AvgKNN', 'Accuracy')



values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_ctg_15, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_ctg_15, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_ctg_15, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_ctg_15, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_ctg_15, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_ctg_15, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_ctg_15, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_ctg_15, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_ctg_15, '1-RE3WC', '10-AvgKNN', 'F1-Score')



/home/baharali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [18]:
middle_method(mode='a')


In [19]:
check_statistical_significance(df_pima_15, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_pima_15, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_pima_15, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_pima_15, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_pima_15, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_pima_15, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_pima_15, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_pima_15, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_pima_15, '1-RE3WC', '10-AvgKNN', 'Accuracy')



values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_pima_15, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_pima_15, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_pima_15, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_pima_15, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_pima_15, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_pima_15, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_pima_15, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_pima_15, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_pima_15, '1-RE3WC', '10-AvgKNN', 'F1-Score')


end_method(mode='a', tab_name='table_15', percentage='15')


In [20]:
start_method(mode='a')

check_statistical_significance(df_ctg_20, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_ctg_20, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_ctg_20, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_ctg_20, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_ctg_20, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_ctg_20, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_ctg_20, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_ctg_20, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_ctg_20, '1-RE3WC', '10-AvgKNN', 'Accuracy')



values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_ctg_20, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_ctg_20, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_ctg_20, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_ctg_20, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_ctg_20, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_ctg_20, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_ctg_20, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_ctg_20, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_ctg_20, '1-RE3WC', '10-AvgKNN', 'F1-Score')


/home/baharali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [21]:
middle_method(mode='a')

In [22]:
check_statistical_significance(df_pima_20, '1-RE3WC', '2-LOF', 'Accuracy', flag=1)
check_statistical_significance(df_pima_20, '1-RE3WC', '3-LoOP', 'Accuracy')
check_statistical_significance(df_pima_20, '1-RE3WC', '4-ABOD', 'Accuracy')
check_statistical_significance(df_pima_20, '1-RE3WC', '5-KNN', 'Accuracy')
check_statistical_significance(df_pima_20, '1-RE3WC', '6-IF', 'Accuracy')

check_statistical_significance(df_pima_20, '1-RE3WC', '7-CBLOF', 'Accuracy')
check_statistical_significance(df_pima_20, '1-RE3WC', '8-FB', 'Accuracy')
check_statistical_significance(df_pima_20, '1-RE3WC', '9-HBOS', 'Accuracy')
check_statistical_significance(df_pima_20, '1-RE3WC', '10-AvgKNN', 'Accuracy')



values = [['\\cline{2-8}'],
        ['& ' + '\\multicolumn{7}{c|}{} \\\\ [-2.5ex] \\cline{2-8}'],
        ['& ' + '\\multirow{9}{*}{F1-Score}']]
write_to_file(file_path2, np.array(values), mode='a')


check_statistical_significance(df_pima_20, '1-RE3WC', '2-LOF', 'F1-Score', flag=1)
check_statistical_significance(df_pima_20, '1-RE3WC', '3-LoOP', 'F1-Score')
check_statistical_significance(df_pima_20, '1-RE3WC', '4-ABOD', 'F1-Score')
check_statistical_significance(df_pima_20, '1-RE3WC', '5-KNN', 'F1-Score')
check_statistical_significance(df_pima_20, '1-RE3WC', '6-IF', 'F1-Score')

check_statistical_significance(df_pima_20, '1-RE3WC', '7-CBLOF', 'F1-Score')
check_statistical_significance(df_pima_20, '1-RE3WC', '8-FB', 'F1-Score')
check_statistical_significance(df_pima_20, '1-RE3WC', '9-HBOS', 'F1-Score')
check_statistical_significance(df_pima_20, '1-RE3WC', '10-AvgKNN', 'F1-Score')


end_method(mode='a', tab_name='table_20', percentage='20')

In [23]:
print(file_path2)
print("Done")

/home/baharali/Medical_Image/Implementations/1. RE3WC/Tvalue_Pvalue_latex.txt
Done
